# Training and testing a 3D Vanilla Point Cloud Autoencoder

## Demonstration of the implemented 3D point cloud autoencoders

**LICENSE: GPL 3.0**
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or 
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
"""
### Related Publications

T. Rios, B. van Stein, S. Menzel, T. Bäck, B. Sendhoff, P. Wollstadt,
"Feature Visualization for 3D Point Cloud Autoencoders", 
International Joint Conference on Neural Networks (IJCNN), 2020
[https://www.honda-ri.de/pubs/pdf/4354.pdf]

T. Rios, T. Bäck, B. van Stein, B. Sendhoff, S. Menzel, 
"On the Efficiency of a Point Cloud Autoencoder as a Geometric Representation
for Shape Optimization", 2019 IEEE Symposium Series on Computational Intelligence 
(SSCI), pp. 791-798, 2019.
[https://www.honda-ri.de/pubs/pdf/4199.pdf]

S. Saha, S. Menzel, L.L. Minku, X. Yao, B. Sendhoff and P. Wollstadt, 
"Quantifying The Generative Capabilities Of Variational Autoencoders 
For 3D Car Point Clouds", 2020 IEEE Symposium Series on Computational Intelligence 
(SSCI), 2020. (submitted)

### Pre-requisites:
 - Python      3.6.10
 - numpy       1.19.1
 - TensorFlow  1.14.0
 - TFLearn     0.3.2
 - cudatoolkit 10.1.168
 - cuDNN       7.6.5
 - Ubuntu      18.04
 - pandas      1.1.0
 
**Copyright (c)
Honda Research Institute Europe GmbH**

Authors: Thiago Rios <thiago.rios@honda-ri.de>



## Step 1: Data set generation
Training and testing the autoencoder requires a data set of 3D point clouds. For the purposes of the example, we will generate point clouds by generating a sphere with harmonic oscillations in the surface. We start to generate the data set by importing the necessary libraries and defining the functions.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd

## Script for handling the exit() command in the scripts
import sys
from io import StringIO
from IPython import get_ipython


class IpyExit(SystemExit):
    def __init__(self):
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup
def ipy_exit():
    raise IpyExit
if get_ipython():    # ...run with IPython
    exit = ipy_exit  # rebind to custom exit
else:
    exit = exit      # just make exit importable


# Sphere
def sphere(phi, theta, fphi, ftheta, x0, y0):
    ## point cloud array
    pc = np.zeros((theta.shape[0], 3))
    
    ## Sphere
    # radius
    R = 1 + 0.25*np.cos(ftheta*theta) + 0.25*np.cos(fphi*phi)
    # points
    pc[:,0] = x0 + R*np.sin(theta)*np.cos(phi)
    pc[:,1] = y0 + R*np.sin(theta)*np.sin(phi)
    pc[:,2] = R*np.cos(theta)    
    return(pc)
    

Now we can systematically generate point clouds mixing the three functions. For the purposes of the example, we will generate 2500 point clouds with oscillations in the surface in the range of [0, 1.0] Hz.

In [ ]:
## X-Y space
phi,theta = np.meshgrid(np.linspace(0,2*np.pi,50), np.linspace(0,np.pi,50))
## Sample frequencies
# Define random seed
np.random.seed(0)
# Sample frequencies
fphi = np.random.random(2500)*1*2*np.pi
ftheta = np.random.random(2500)*1*2*np.pi

## Generate point clouds
import os
# data set directory
data_dir = "./benchmark_pointclouds"
if not os.path.exists(data_dir): os.system("mkdir {}".format(data_dir))
# rastrigin
for i in range(2500):
    # evaluate data
    pc = sphere(phi.flatten(), theta.flatten(), fphi[i], ftheta[i], 0, 0)
    pd.DataFrame(pc).to_csv("{}/pc_{}.xyz".format(data_dir, i),\
                            header=None, index=None, sep=" ")

In order to train the vanilla point cloud autoencoder on the generated data, we need to call the script "pcae_training.py" in the _include_ directory. In the terminal, the command line is
`python [path_to_pcae_training.py] --N [pc_size] --LR [lr_size] --GPU [gpu_id] --i [data_dir_path]`. However, for running the script in the Jupyter Notebook, the command is preceeded by `%run -i`.

For training the variational point cloud autoencoder, the script is the `vpcae_training.py`, which should be called with the same inputs options.

In [ ]:
%run -i inputscripts/pcae_training.py --N 1024 --LR 128 --GPU 0 --i benchmark_pointclouds 

Once the model is trained, we can test the model with respect to the quality of the point cloud reconstruction. The first step is to calculate the reconstruction loss (Chamfer Distance) on a set of point clouds, wich is specified in a text file. This test is performed with the command `python evaluate_loss.py --N [point_cloud_size] --LR [latent_representation_size] --GPU [gpu_id] --i [list_with_pointclouds_path]`, and as a list of test point clouds, we will use the test set, specified in `Network_pcae_N1024_LR_10/geometries_testing.csv`. The results of the test are stored in the file `Network_pcae_N1024_LR_10/pcae_test/reconstruction_losses.dat`

For the remaining scripts, **if the Variational autoencoder is used, the option --VAE True needs to be added.**

In [ ]:
%run -i inputscripts/evaluate_loss.py --N 1024 --LR 128 --GPU -1 --i Network_pcae_N1024_LR128/geometries_testing.csv

The second test is to generate point clouds from samples in the latent space, which are described in a text file. Since the latent space is constrained to the interval $[-1,1]^L$, where _L_ is the dimensionality of the latent space, we can generate a set of random samples and recover the point clouds. First, to illustrate how it works, we will take ten samples from the test set, calculated in the previous step, to generate point clouds.

In [ ]:
# Sample the latent space
file_test = np.array(pd.read_csv("Network_pcae_N1024_LR128/pcae_test/reconstruction_losses.dat"))
lr_data = file_test[0:10,1:-1]
# Save lr data
pd.DataFrame(lr_data).to_csv("lr_data.dat", header=None, index=None)

This evaluation is performed by calling the command `python pointcloud_generator.py --N [point_cloud_size] --LR [latent_representation_size] --GPU [gpu_id] --i [list_with_latent_variables]`. The script reconstruct the point clouds and sotre them as `.xyz` files and `.html` scatter plots in `Network_pcae_N1024_LR_128/point_cloud_generation`.

In [ ]:
%run -i inputscripts/pointcloud_generator.py --N 1024 --LR 128 --GPU -1 --i lr_data.dat 

Now, we will take random samples from the latent space. Since the autoencoder usually does not distribute data uniformly over the latent space, some of the samples might yield noisy os collapsed point clouds.

In [ ]:
# Generate samples
lr_data2 = lr_data[0,:]*(np.reshape(np.linspace(1,0,10), (-1,1)))\
                         + lr_data[4,:]*(np.reshape(np.linspace(0,1,10), (-1,1)))
# Save lr data
pd.DataFrame(lr_data2).to_csv("lr_data2.dat", header=None, index=None)
%run -i inputscripts/pointcloud_generator.py --N 1024 --LR 128 --GPU -1 --i lr_data2.dat 